In [82]:
import cv2
import mediapipe as mp
import time
from collections import namedtuple

In [83]:
Point = namedtuple('Point', 'x y')

In [84]:
class Line:
    def __init__(self, P, Q):
        self.a = P.y - Q.y
        self.b = Q.x - P.x
        self.c = P.x*(Q.y - P.y) - P.y*(Q.x - P.x)

    def substitute(self, R):
        return self.a*R.x + self.b*R.y + self.c

In [85]:
opened = False
opentime = 0
closetime = 0
checkopen = True
checkclose = False
open = True
status = "CLOSED"

In [86]:
cap = cv2.VideoCapture(cv2.CAP_DSHOW)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
pTime = 0
cTime = 0
morse = ""

while True:
    ret, frame = cap.read()
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:

            A = Point(int(handLms.landmark[0].x*w), int(handLms.landmark[0].y*h))
            B = Point(int(handLms.landmark[5].x*w), int(handLms.landmark[5].y*h))
            C = Point(int(handLms.landmark[17].x*w), int(handLms.landmark[17].y*h))
            P = Point(int(handLms.landmark[12].x*w), int(handLms.landmark[12].y*h))

            l1 = Line(A, B)
            l2 = Line(B, C)
            l3 = Line(C, A)

            if l1.substitute(P) > 0 and l2.substitute(P) > 0 and l3.substitute(P) > 0:
                status = "CLOSED"
            else:
                status = "OPEN"
            mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)
            cv2.putText(frame, status, (10, 300), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 3, 1)

    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    cv2.putText(frame, str(int(fps)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 1.7, (0, 0, 255), 3, 1)

    if status == "OPEN" and checkopen:
        opened = True
    elif status == "CLOSED" and checkclose:
        open = False
        opened = False
        checkclose = False

    if opened:
        opentime = time.time()
        opened = False
        checkopen = False
        checkclose = True
        open = True
    elif not open:
        open = True
        closetime = time.time()
        timeopen = closetime - opentime

        if timeopen < 0.7:
            morse += "."
        else :
            morse += "_"
        checkopen = True


    cv2.imshow("Video", frame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()
print(morse)

.._.
